<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module_1/images/SN_web_lightmode.png" width="300"> 
</center>


<h1>Data Wrangling with Regular Expressions</h1>

Estimated time needed: **40** minutes


## Lab Overview:

In the previous data collection labs, you collected some raw datasets from several different sources. In this lab, you need to perform data wrangling tasks in order to improve data quality.


You will again use regular expressions, along with the `stringr` package (part of `tidyverse`), to clean up the bike-sharing systems data that you previously web scraped from the wiki page: 

https://en.wikipedia.org/wiki/List_of_bicycle-sharing_systems

<a href="https://cognitiveclass.ai/">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/module_1/images/l2-list-bike-sharing-systems.png" width="800" align="center">
</a>


One typical challenge of web scraping is that data extracted from HTML pages may contain unnecessary or inconsistently fomatted information.  
For example:
 - Textual annotations in numeric fields: `1000 (Updated with 1050)`
 - Attached reference links: `Bike sharing system [123]`
 - Inconsistent data formats: `Yes` and `Y` for the logical value `TRUE` or `2021-04-09` and `Apr 09, 2021` for the same date
 - HTML style tags: `<span style="color:blue">Bike sharing system</span>`
 - Special characters: `&nbsp` for a white space
 
Many more such examples of noise may be encountered in real-world scraped data and most of such text related noises could be handled by regular expressions.


To summarize, you will be using `stringr` (part of `tidyverse`) and regular expressions to perform the following data wrangling tasks:
  - TASK: Standardize column names for all collected datasets
  - TASK: Remove undesired reference links from the scraped bike-sharing systems dataset
  - TASK: Extract only the numeric value from undesired text annotations


Let's begin by importing the libraries you will use for these data wrangling tasks. 

Please note that the require("tidyverse")" command is commented here as the tidyverse package is already pre-installed in this lab environment. However, if you are executing this lab local R-Studio on your system then install this package first and then load the package.


In [1]:
# Check whether you need to install the `tidyverse` library
# require("tidyverse")
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
√ ggplot2 3.1.1       √ purrr   0.3.2  
√ tibble  2.1.1       √ dplyr   0.8.0.1
√ tidyr   0.8.3       √ stringr 1.4.0  
√ readr   1.3.1       √ forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


## TASK: Standardize column names for all collected datasets 


In the previous data collection labs, you collected four datasets in csv format:
- `raw_bike_sharing_systems.csv`:  A list of active bike-sharing systems across the world
- `raw_cities_weather_forecast.csv`: 5-day weather forecasts for a list of cities, from OpenWeather API
- `raw_worldcities.csv`: A list of major cities' info (such as name, latitude and longitude) across the world
- `raw_seoul_bike_sharing.csv`: Weather information (Temperature, Humidity, Windspeed, Visibility, Dewpoint, Solar radiation, Snowfall, Rainfall), the number of bikes rented per hour, and date information, from Seoul bike-sharing systems


*Optional:* If you had some difficulties finishing the data collection labs, you may download the datasets directly from the following URLs:


In [2]:
# Download raw_bike_sharing_systems.csv
url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_bike_sharing_systems.csv"
download.file(url, destfile = "raw_bike_sharing_systems.csv")

# Download raw_cities_weather_forecast.csv
url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_cities_weather_forecast.csv"
download.file(url, destfile = "raw_cities_weather_forecast.csv")

# Download raw_worldcities.csv
url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_worldcities.csv"
download.file(url, destfile = "raw_worldcities.csv")

# Download raw_seoul_bike_sharing.csv
url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_seoul_bike_sharing.csv"
download.file(url, destfile = "raw_seoul_bike_sharing.csv")

To improve dataset readbility by both human and computer systems, we first need to standardize the column names of the datasets above using the following naming convention:
 - Column names need to be UPPERCASE
 - The word separator needs to be an underscore, such as in `COLUMN_NAME`


You can use the following dataset list and the `names()` function to get and set each of their column names, and convert them according to our defined naming convention.


In [3]:
dataset_list <- c('raw_bike_sharing_systems.csv', 'raw_seoul_bike_sharing.csv', 'raw_cities_weather_forecast.csv', 'raw_worldcities.csv')

_TODO_: Write a `for` loop to iterate over the above datasets and convert their column names 


In [4]:
for (dataset_name in dataset_list){
    # Read dataset
    dataset <- read_csv(dataset_name, col_types=cols())
    # Standardized its columns:
    # Convert all column names to uppercase
    names(dataset) <- map(names(dataset),toupper)
    
    # Replace any white space separators by underscores, using the str_replace_all function    
    for( idx in c( 1:length(names(dataset)) ) ){
        name <- names(dataset)[[idx]]
        name <- gsub('(^ | $)','',name)
        name <- gsub(' ','_',name)
        names(dataset)[idx] <- name
    }
    
    # Save the dataset 
    write.csv(dataset, dataset_name, row.names=FALSE)
}


Warning message:
"Must use a character vector as names.
This warning is displayed once per session."

_TODO_: Read the resulting datasets back and check whether their column names follow the naming convention


In [5]:
for (dataset_name in dataset_list){
    # Print a summary for each data set to check whether the column names were correctly converted
    df<-read_csv(dataset_name, col_types=cols())
    str(df)
}

Classes 'spec_tbl_df', 'tbl_df', 'tbl' and 'data.frame':	480 obs. of  10 variables:
 $ COUNTRY        : chr  "Albania" "Argentina" "Argentina" "Argentina" ...
 $ CITY           : chr  "Tirana" "Mendoza" "San Lorenzo, Santa Fe" "Buenos Aires" ...
 $ NAME           : chr  "Ecovolis" "Metrobici" "Biciudad" "Ecobici" ...
 $ SYSTEM         : chr  NA NA "Biciudad" "Serttel Brasil" ...
 $ OPERATOR       : chr  NA NA NA "Bike In Baires Consortium.[10]" ...
 $ LAUNCHED       : chr  "March 2011" "2014" "27 November 2016" "2010" ...
 $ DISCONTINUED   : chr  NA NA NA NA ...
 $ STATIONS       : chr  "8" "2" "8" "400" ...
 $ BICYCLES       : chr  "200" "40" "80" "4000" ...
 $ DAILY_RIDERSHIP: chr  NA NA NA "21917" ...
 - attr(*, "spec")=
  .. cols(
  ..   COUNTRY = col_character(),
  ..   CITY = col_character(),
  ..   NAME = col_character(),
  ..   SYSTEM = col_character(),
  ..   OPERATOR = col_character(),
  ..   LAUNCHED = col_character(),
  ..   DISCONTINUED = col_character(),
  ..   STATIONS =

## Process the web-scraped bike sharing system dataset 


By now we have standardized all column names. Next, we will focus on cleaning up the values in the web-scraped bike sharing systems dataset.


In [6]:
# First load the dataset
bike_sharing_df <- read_csv("raw_bike_sharing_systems.csv", col_types=cols())

In [7]:
# Print its head
head(bike_sharing_df)

COUNTRY,CITY,NAME,SYSTEM,OPERATOR,LAUNCHED,DISCONTINUED,STATIONS,BICYCLES,DAILY_RIDERSHIP
Albania,Tirana,Ecovolis,NA,NA,March 2011,NA,8,200,NA
Argentina,Mendoza,Metrobici,NA,NA,2014,NA,2,40,NA
Argentina,"San Lorenzo, Santa Fe",Biciudad,Biciudad,NA,27 November 2016,NA,8,80,NA
Argentina,Buenos Aires,Ecobici,Serttel Brasil,Bike In Baires Consortium.[10],2010,NA,400,4000,21917
Argentina,Rosario,Mi Bici Tu Bici[11],NA,NA,2 December 2015,NA,47,480,NA
Australia,Melbourne[12],Melbourne Bike Share,PBSC & 8D,Motivate,June 2010,30 November 2019[13],53,676,NA


Even from the first few rows, you can see there is plenty of undesireable embedded textual content, such as the reference link included in `Melbourne[12]`.


In this project, let's only focus on processing the following revelant columns (feel free to process the other columns for more practice):
 - `COUNTRY`: Country name 
 - `CITY`: City name
 - `SYSTEM`: Bike-sharing system name
 - `BICYCLES`: Total number of bikes in the system


In [8]:
# Select the four columns
sub_bike_sharing_df <- bike_sharing_df %>% select(COUNTRY, CITY, SYSTEM, BICYCLES)

Let's see the types of the selected columns


In [9]:
sub_bike_sharing_df %>% 
    summarize_all(class) %>%
    gather(variable, class)

variable,class
COUNTRY,character
CITY,character
SYSTEM,character
BICYCLES,character


They are all interpreted as character columns, but we expect the `BICYCLES` column to be of numeric type. Let's see why it wasn't loaded as a numeric column - possibly some entries contain characters. Let's create a simple function called `find_character` to check that.


In [10]:
# grepl searches a string for non-digital characters, and returns TRUE or FALSE
# if it finds any non-digital characters, then the bicyle column is not purely numeric
find_character <- function(strings) grepl("[^0-9]", strings)

Let's try to find any elements in the `Bicycles` column containing non-numeric characters.


In [11]:
sub_bike_sharing_df %>% 
    select(BICYCLES) %>% 
    filter(find_character(BICYCLES)) %>%
    slice(0:10)

BICYCLES
4115[22]
310[59]
500[72]
[75]
180[76]
600[77]
[78]
initially 800 (later 2500)
100 (220)
370[114]


As you can see, many rows have non-numeric characters, such as `32 (including 6 rollers) [162]` and `1000[253]`. This is actually very common for a table scraped from Wiki when no input validation is enforced. 

Later, you will use regular expressions to clean them up.


Next, let's take a look at the other columns, namely `COUNTRY`, `CITY`, and `SYSTEM`, to see if they contain any undesired reference links, such as in `Melbourne[12]`.


In [12]:
# Define a 'reference link' character class, 
# `[A-z0-9]` means at least one character 
# `\\[` and `\\]` means the character is wrapped by [], such as for [12] or [abc]
ref_pattern <- "\\[[A-z0-9]+\\]"
find_reference_pattern <- function(strings) grepl(ref_pattern, strings)

In [13]:
# Check whether the COUNTRY column has any reference links
sub_bike_sharing_df %>% 
    select(COUNTRY) %>% 
    filter(find_reference_pattern(COUNTRY)) %>%
    slice(0:10)

COUNTRY


Ok, looks like the `COUNTRY` column is clean. Let's check the `CITY` column.


In [14]:
# Check whether the CITY column has any reference links
sub_bike_sharing_df %>% 
    select(CITY) %>% 
    filter(find_reference_pattern(CITY)) %>%
    slice(0:10)

CITY
Melbourne[12]
Brisbane[14][15]
Lower Austria[18]
Namur[19]
Brussels[21]
Salvador[23]
Belo Horizonte[24]
Joao Pessoa[25]
(Pedro de) Toledo[26]
Rio de Janeiro[27]


Hmm, looks like the `CITY` column has some reference links to be removed. Next, let's check the `SYSTEM` column.


In [15]:
# Check whether the System column has any reference links
sub_bike_sharing_df %>% 
    select(SYSTEM) %>% 
    filter(find_reference_pattern(SYSTEM)) %>%
    slice(0:10)

SYSTEM
EasyBike[58]
4 Gen.[61]
3 Gen. SmooveKey[113]
3 Gen. Smoove[141][142][143][139]
3 Gen. Smoove[179]
3 Gen. Smoove[181]
3 Gen. Smoove[183]


So the `SYSTEM` column also has some reference links.


After some preliminary investigations, we identified that the `CITY` and `SYSTEM` columns have some undesired reference links, and the `BICYCLES` column has both reference links and some 
textual annotations.

Next, you need to use regular expressions to clean up the unexpected reference links and text annotations in numeric values.


# TASK: Remove undesired reference links using regular expressions


_TODO:_ Write a custom function using `stringr::str_replace_all` to replace all reference links with an empty character for columns `CITY` and `SYSTEM`


In [16]:
# remove reference link
remove_ref <- function(strings) {
    ref_pattern <- "\\[[\\w]+\\]"
    strings <- str_replace_all(strings, ref_pattern, '')
    strings <- str_replace_all(strings, '(^ | $)', '')
    return(strings)
}

_TODO:_ Use the `dplyr::mutate()` function to apply the `remove_ref` function to the `CITY` and `SYSTEM` columns


In [17]:
sub_bike_sharing_df <- sub_bike_sharing_df %>% 
                        mutate(CITY=remove_ref(CITY), SYSTEM=remove_ref(SYSTEM), BICYCLES=remove_ref(BICYCLES))

_TODO:_ Use the following code to check whether all reference links are removed:


In [18]:
sub_bike_sharing_df %>% 
    select(CITY, SYSTEM, BICYCLES) %>% 
    filter(find_reference_pattern(CITY) | find_reference_pattern(SYSTEM) | find_reference_pattern(BICYCLES))

CITY,SYSTEM,BICYCLES


# TASK: Extract the numeric value using regular expressions


_TODO:_ Write a custom function using `stringr::str_extract` to extract the first digital substring match and convert it into numeric type For example, extract the value '32' from `32 (including 6 rollers) [162]`.


In [19]:
# Extract the first number
extract_num <- function(columns){
    digitals_pattern <- "^[\\d\\.]+"
    str_extract(columns, "^[0-9]+\\.{0,1}[0-9]*")
    columns <- as.numeric(columns)
    
    return(columns)
} 

_TODO:_ Use the `dplyr::mutate()` function to apply `extract_num` on the `BICYCLES` column


In [20]:
# Use the mutate() function on the BICYCLES column
sub_bike_sharing_df <- sub_bike_sharing_df %>% mutate(BICYCLES=extract_num(BICYCLES))

Warning message in extract_num(BICYCLES):
"강제형변환에 의해 생성된 NA 입니다"

_TODO:_ Use the summary function to check the descriptive statistics of the numeric `BICYCLES` column


In [21]:
summary(sub_bike_sharing_df$BICYCLES)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
      5     100     350    2032    1400   78000      83 

_TODO:_ Write the cleaned bike-sharing systems dataset into a csv file called `bike_sharing_systems.csv`


In [22]:
# Write dataset to `bike_sharing_systems.csv`
write.csv(sub_bike_sharing_df, file='bike_sharing_systems.csv', row.names=FALSE)

# References:


If you need to refresh your memory about regular expressions, please refer to this good Regular Expression cheat sheet:

<a href="https://www.rstudio.com/wp-content/uploads/2016/09/RegExCheatsheet.pdf" target="_blank">Basic Regular Expressions in R</a>


# Next Steps


Great! Now you have cleaned up the bike-sharing system dataset using regular expressions. Next, you will use other `tidyverse` functions to perform data wrangling on the bike-sharing demand dataset.


## Authors

<a href="https://www.linkedin.com/in/yan-luo-96288783/" target="_blank">Yan Luo</a>


### Other Contributors

Jeff Grossman


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description           |
| ----------------- | ------- | ---------- | ---------------------------- |
| 2021-04-08        | 1.0     | Yan        | Initial version created      |
|                   |         |            |                              |
|                   |         |            |                              |

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
